In [ ]:
import rdkit
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.SimDivFilters.rdSimDivPickers import MaxMinPicker
from itertools import chain
from rdkit.Chem import Descriptors
import pickle
import itertools

In [ ]:
#class for containing all the variables individual needs in non-dominated
#sorting algorithim, takes SMILE string in constructor
class Indidividual:

  def __init__(self, smi):
    self.domination_count = 0
    self.dominated_solutions = []
    self.rank = None
    self.front = None
    self.SMILE = smi
    self.mol = Chem.MolFromSmiles(smi)
    self.fps = AllChem.GetMorganFingerprintAsBitVect(self.mol,2,nBits=1024)
    self.crowding_distance = None

In [ ]:
class GA_Utils:  
 
  def fast_nondominated_sort(self, population):
    fronts = [[]]
    for individual in population:
      
      #settting default values for domination_count and dominated_solutions
      individual.domination_count = 0
      individual.dominated_solutions = []
      
      #comparing every other individual to individual to see which dominates
      #and which doenst
      for other_individual in population:
        
        #if individual dominates the other, add the other to dominated solutions
        if self.check_dominate(individual, other_individual):
          individual.dominated_solutions.append(other_individual)
        
        #if individual is dominated by the other, add 1 to domination count
        elif self.check_dominate(other_individual, individual):
          individual.domination_count += 1
          
      #add all individuals with domination_count 0 to front 1
      if individual.domination_count == 0:
        individual.front = 0
        fronts[0].append(individual)
      
    i = 0
    while len(fronts[i]) > 0:
      temp = []
      
      for individual in fronts[i]:
        
        #removing 1 from domination count of each other individual dominated by
        #individual 
        for other_individual in individual.dominated_solutions:
          other_individual.domination_count -= 1
          
          #checking if other individual now has domination count 0
          if other_individual.domination_count == 0:
            
            #adding individiual to next front
            other_individual.rank = i+1
            temp.append(other_individual)
      i = i+1
      
      #adding newly created front to list of current fronts
      fronts.append(temp)

    return fronts
    

  #functions for calculating chemical properties of a molecule in SMILE string
  def get_mol_mass(self, mol):
    return Descriptors.MolWt(mol)

  def get_hydrogen_bond_donors(self, mol):
    return Descriptors.NumHDonors(mol)

  def get_hydrogen_bond_acceptors(self, mol):
    return Descriptors.NumHAcceptors(mol)

  def get_log_p(self, mol):
    return Descriptors.MolLogP(mol)

  #checking if an individual dominates another
  def check_dominate(self, indiv1, indiv2):
    and_condition = True
    or_condition = False
    
    #comparing every individual objective of each individual to determine
    #whether or not indiv1 dominates indiv2
    for first, second in zip(self.calculate_objectives(indiv1), self.calculate_objectives(indiv2)):
      and_condition = and_condition and first <= second
      or_condition = or_condition or first < second
    return (and_condition and or_condition)

  #if only part of a front can be kept, this function ranks the members in the
  #front and chooses the best x members
  def calculate_crowding_distance(self, front, picks):
    new_front = []
    
    #rdkit function for picking most diverse molecules out of set of molecules
    picker = MaxMinPicker()
    
    #checking if front is not empty
    if len(front) > 0 and picks < len(front):
      
      
      pick_indices = picker.LazyPick(self.distance, len(front), picks)
      
      #adding chosen molecules to create new front
      for i in pick_indices:
        new_front.append(front[pick_indices])
      
      return new_front
    
    else:
      print("Front is length 0 or picks exceeds front length")

  #calculating similarity between two molecules
  def distance(self, indiv1,indiv2):
    return 1-DataStructs.DiceSimilarity(indiv1.fps,indiv2.fps)
    
  #calculating each objective
  def calculate_objectives(self, indiv):
    mol = indiv.mol
    
    #calculating chemical properties of molecule
    mol_mass = self.get_mol_mass(mol)
    hyd_bond_donors = self.get_hydrogen_bond_donors(mol)
    hyd_bond_acceptors = self.get_hydrogen_bond_acceptors(mol)
    log_p = self.get_log_p(mol)

    obj_mass = 0
    obj_donors = 0
    obj_acceptors = 0
    obj_log_p = 0

    #if objectives are within lipinskis RO5, objectives are set to 0
    if mol_mass <= 500:
      obj_mass = 0
    else:
      obj_mass = mol_mass - 500

    if hyd_bond_donors <= 5:
      obj_donors = 0
    else:
      obj_donors -= 5

    if hyd_bond_acceptors <= 5:
      obj_acceptors = 0
    else:
      obj_acceptors -= 5

    if log_p <= 5:
      log_p = 0
    else:
      obj_log_p -= 5

    #retrning calculated objectives
    return obj_mass, obj_donors, obj_acceptors, obj_log_p

In [ ]:
class run_GA():  
  def __init__(self, SMILES, population_kept):
    
    #creating population from list of SMILE strings
    self.population = self.create_population(SMILES)
    
    self.population_kept = population_kept
    self.population_removed = len(self.population) - self.population_kept
    self.utils = GA_Utils()

  def run_sort(self):
    #create fronts using non dominated sort
    fronts = self.utils.fast_nondominated_sort(self.population)
    molecules = []
    
    #fronts are technically backwards, worst solutions are in front 1 so fronts
    #have to be reversed so best solutions are in front 1
    fronts.reverse()

    length = 0
    for i, front in enumerate(fronts):
      #calculating length of fronts up to i
      length += len(front)
      front_keep = len(front) - (length - self.population_kept)
      
      #checking if some of molecules in front need to be discarded, 
      #if so, storing front number
      if(length > self.population_kept):
        edge_front = fronts[i]
        front_number = i
      
      #if front lies perfectly within the number of molecules which need to be
      #kept, returns list of indiviudals in kept fronts 
      if(length == self.population_kept):
        front_number = i
        fronts = fronts[:front_number]
        return list(chain.from_iterable(fronts)) 

    
    #for front on edge, calculating which individuals in that front should be
    #kept and which should be goten rid of
    fronts[front_number] = self.utils.calculate_crowding_distance(fronts[front_number], front_keep)
    
    #returning list of individuals in each kept front
    return list(chain.from_iterable(fronts[:front_number]))

  #creating population from list of SMILE
  def create_population(self, SMILES):
    population = []
    for smi in SMILES:
      population.append(Indidividual(smi))
      
    return population
      

In [ ]:
generated_malaria = []
with open('generated_malaria', 'rb') as fp:
    generated_malaria = pickle.load(fp)

In [ ]:
obj = run_GA(generated_malaria, 75000)
fronts = obj.run_sort()

In [ ]:
with open('ranked_molecules', 'wb') as fp:
  pickle.dumb(fronts, fp)